# Prédiction de consommation électrique  avec `scikit-learn` 

Dans ce TP, nous allons étudier la consommation électrique en France métropole entre janvier 2016 et mai 2023, à partir des données provenant du réseau de transport d’électricité français (RTE), disponibles sur le site https://analysesetdonnees.rte-france.com/.

Le fichier `data/data.csv` (ou `data/data.pkl`) a été obtenu en agrégeant les données contenues dans les fichiers suivants : 
- `rte.csv` : consommation d'électricité relevée toutes les demi-heures
- `calendar.csv` : informations sur le temps
- `meteo.csv` : informations liées à la météo, notamment l'emplacement des stations et les différents relevés (température, nébulosité, humidité, vitesse du vent, précipitation), pris toutes les 3 heures. 

Pour les détails sur la génération de ce fichier, voir le notebook `day2_preparation_data.ipynb`.

# EX 1 : Import des données

Les données agrégées se trouvent dans le fichier `data/data.pkl`.

- Importer les données sous forme d'un `DataFrame` pandas que l'on nomme `data`
- Visualiser les premières et dernières lignes des données
- Afficher la taille du `DataFrame`
- Afficher les noms des premières 17 colonnes et ensuite des colonnes restantes
- Faire de même pour le type de données présentes dans ces colonnes
- Vérifier que le `DataFrame`ne contient pas de données manquantes
- Convertir les variables `Mois`, `Jour`, `JourFerieType` en variables dummies

# Analyse des données

Observons le tableau `data` des données agregées.

## EX 2 : Saisonnalité

Afficher le graphe des consommations éléctriques en fonction du temps. Observez-vous une saisonnalité ? 

## Profils de consommation

En utilisant les fonctions `groupby` et `agg` de pandas on peut regrouper les données en faisant la moyenne des consommations par mois, par jour, ou par demi-heure, de sorte à obtenir des profils type de consommation pour une année/un mois/une semaine/un jour.

Par exemple, la moyenne des consommations à l'échelle des années montre des plus fortes consommations pendant les mois d'hiver et plus faibles pendant l'été. :

In [ ]:
# Plot the annual average consumption 
monthly_avg = data.groupby(["DemiHeure", "Mois", "MJour"]).agg(Consommation=("Consommation", "mean"), DateTime =("DateTime" ,"first")).reset_index()
plt.figure(figsize=(12, 6))
ax = sns.lineplot(x="DateTime", y="Consommation", data=monthly_avg, hue="Mois", legend=False) 
plt.xlabel("") 
plt.ylabel("")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
plt.title("Average Consumption")
plt.show()

### Exercice 3

- Afficher le profil de consommation hebdomadaire et journalier
- Afficher le profil de consommation journalier par jour de la semaine

## Comportements particuliers

Il est possible de mettre en évidence des petites anomalies dues au jours fériés ou au printemps 2020. 

La consommation du mois de mai 2017, en mettant en évidence les jours féries (le 1er mai 2017 était un lundi) :

In [ ]:
# Plot the consumption for May 2017
may_2017_data = data[(data['Annee'] == 2017) & (data['Mois'] == 'mai')]
may_2017_ferie = may_2017_data[may_2017_data['JourFerie'] == 1]
may_2017_ferie['MJour'].unique()

plt.figure(figsize=(12, 6))
plt.plot(may_2017_data['DateTime'], may_2017_data['Consommation'], color='blue')
for i in may_2017_ferie['MJour'].unique():
    plt.plot(may_2017_ferie[may_2017_ferie['MJour'] == i]['DateTime'], may_2017_ferie[may_2017_ferie['MJour'] == i]['Consommation'], color='black')
plt.xlabel('')
plt.ylabel('')
plt.title('Consumption in May 2017')
plt.show()

#### EX 4
- Comparer la consommation moyenne de l'année 2020 à celle du reste des données

# Modèles : entraînement, prédiction et erreur 

## EX 5 : Séparation du jeu de données en train et test

Séparer le jeu de données en deux pour obtenir le jeu d'entraînement et le jeu de test tels que 

- `X_train` et `Y_train` contiennent 50% du jeu de données pour l'apprentissage du modèle  
- `X_test` et `Y_test` contiennent les 50% restants pour le test.

On pourra utiliser la  fonction `train_test_split` du module `sklearn.model_selection`.

## Naive predictor

Pour pouvoir comparer les différents modèles, nous allons construire un prédicteur naïf qui renvoie la moyenne des consommations (une constante). Ce prédicteur trivial représente le point de départ, à partir duquel nous allons nous améliorer.

Pour évaluer les performances des modèles, nous allons calculer l'erreur MAPE et l'erreur RMSE des prédicteurs sur le jeu de test et sur celui de train.

Pour rappel, si $\hat Y_t$ est la valeur prédite par le modèle et $Y_t$ la consommation réalisée :

MAPE = **mean absolute percentage error** : c'est une erreur de prédiction *relative*:
$${\displaystyle {\mbox{MAPE}}={\frac {1}{n}}\sum _{t=1}^{n}\left|{\frac {Y_{t}-\hat Y_{t}}{Y_{t}}}\right|}
$$
En pratique, on préfère diviser par $|Y_{t}+\varepsilon|$ pour éviter des division par zéro.

RMSE = **root mean squared error** : c'est une erreur de prédiction *absolue*, sensible aux outliers:
$${\displaystyle {\mbox{RMSE}}=\sqrt{\frac 1 n \sum _{t=1}^{n}\left(Y_{t}-\hat Y_{t}\right)^2}}
$$

### EX 6

- Construire un prédicteur naïf qui renvoie la moyenne des consommations sur le jeu de train et sur celui de test.
- Calculer les erreurs MAPE et RMSE de ce prédicteur naïf (sur le jeu de train et le jeu de test)
- Stocker ces erreurs dans un DataFrame `df_errors`, qui va servir à stocker les erreurs des différents prédicteurs que nous allons mettre en oeuvre.


On pourra utiliser les fonctions `mean_absolute_percentage_error` et `root_mean_squared_error` du module `sklearn.metrics`.

## Fonctions utiles

Tous les modèles de régression de la librairie scikit-learn ont une méthode `fit` qui permet d'entraîner le modèle et une méthode `predict` qui calcule la valeur prédite.

Nous allons comparer plusieurs méthodes et pour cela il sera utile de définir une fonction `fit_and_predict_error` qui, étant donné un modèle `model`, et un jeu de données `x_train`,`y_train`, `x_test`, `y_test`, entraîne le modèle en appelant la méthode `fit` du modèle sur le jeu d'entraînement, calcule la prédiction en appelant la méthode `predict` du modèle à la fois sur le jeu de test et sur le jeu de train, et calcule les erreurs MAPE et RMSE sur les 2 jeux.

Cette fonction renvoie un dictionnaire contenant les prédictions calculées, `train` et `test`, et les erreurs MAPE et RMSE pour les jeux de train et de test.

Pour stocker l'information sur les erreurs dans le tableau des erreurs `df_errors`, nous définissons également une fonction qui permet de le faire rapidement. 

In [ ]:
def add_error(model_out, model_name, df):
    return df._append({'Model' : model_name, 
                  'MAPE test' : model_out['mape_test'], 
                  'RMSE test' : model_out['rmse_test'], 
                  'MAPE train' : model_out['mape_train'], 
                  'RMSE train' : model_out['rmse_train'], 
                  'CPU time' : model_out['time']}, ignore_index=True)

## Regression linéaire

Nous allons en première approche utiliser des modèles linéaires pour apprendre et prévoir les données de consommation.  

A l'aide de la librairie `sklearn.linear_model`, nous pouvons mettre en place plusieurs modèles de regression linéaire pour prédire la variable `Consommation` en fonction de différentes caractéristiques.

Un premier modèle linéaire (modèle 1) utilise les données de température comme covariables.

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
##### Model
linear_pipe = make_pipeline(StandardScaler(), LinearRegression(fit_intercept=True))

# Model 1 : Linear regression on temperature
linear_temp = fit_and_predict_error(linear_pipe, X_train[['Temperature']], Y_train, X_test[['Temperature']], Y_test)
complete_linear_temp = linear_pipe.predict(data[['Temperature']])
df_errors = add_error(linear_temp, 'Linear on Temperature', df_errors)


### EX 7

Mettre en place plusieurs modèles de regression linéaire pour prédire la variable `Consommation` en fonction des caractéristiques :

    2. Données méteo
    3. Données méteo et `PositionDansAnnee`, `JourFerie`, `DemiHeure`
    4. Les caractéristiques ['Temperature', 'Nebulosity', 'Humidity', 'WindSpeed', 'Precipitation', 'PositionDansAnnee', 'DemiHeure', 'JourFerie',  'Vacances', 'MJour', 'Annee', 'is.2020']
    5. Toutes les caractéristiques

- Lequel de ces modèles donne les meilleurs résultats ? 

- Afficher, pour les modèles naïf, 1, 4 et 5, la consommation réelle et la consommation prédite sur les données complètes, sur un même graphe (il y a donc 4 graphes, un par modèle). 

Nous allons dorenavant travailler uniquement sur deux ensembles de caractéristiques :
- Les caractéristiques réduites  :  ['Temperature', 'Nebulosity', 'Humidity', 'WindSpeed', 'Precipitation', 'PositionDansAnnee', 'DemiHeure', 'JourFerie', 'Vacances', 'MJour', 'Annee', 'is.2020']
- Les caractéristiques complètes

### EX 8

Créer deux jeux de données d'entrainement (et de test) `X_train_S` et `X_train_L` (respectivement `X_test_S` et `X_test_L`) correspondants à ces deux ensembles.

## Lasso and Ridge regression

Essayons d'améliorer les performances en ajoutant une pénalité $\ell_1$ et une pénalité $\ell_2$ à la regression linéaire. 

### EX 9 :  Lasso et Ridge

A l'aide des modules `LassoCV` et `RidgeCV` mettre en place deux estimateurs sur les données réduites et sur les données complètes.

## Arbres de décision -- CART

Les *Classification and Regression Trees* (CART) permettent de prédire la consommation en suivant des simples règles de décision qui dépendent des caractéristiques observées.

Le choix du nombres et du type de caractéristiques utilisées pour construire un arbre, ainsi que sa profondeur influent énormément sur le résultat, les paramètres à indiquer dans sa construction sont à choisir attentivement. 

A noter que dans un contexte d'arbres de décision la normalisation des caractéristiques n'est pas nécessaire.

### EX 10 : Arbre par défaut

Utiliser la classe `DecisionTreeRegressor` du module `sklearn.tree` pour construire un estimateur basé sur un arbre de décision.

- Utiliser initialement les paramètres par défaut.

- Entraîner le modèle sur les données réduites et sur les données complètes.

- Afficher les erreurs. Que remarque-t-on sur les erreurs calculées sur le train set ? Comment le justifier ?

### Exercice : Réduction de profondeur

Retrouver dans la documentation les valeurs par défaut des paramètres d'un arbre de régression suivants :

- max_depth
- min_samples_leaf
- max_features
  
Quelle est la profondeur de l'arbre construit par défaut ? 

Construire un arbre de profondeur 10 et comparer les erreurs avec les modèles précédents.

### Exercice : Visualisation

- Construire un arbre de profondeur 4 et l'entraîner sur les données complètes
- A l'aide de la fonction `plot_tree` du module `tree` de sklearn, visualiser l'arbre de décision.

- Quelles sont les caractéristiques sur lesquelles sont fait les splits de cet arbre ? 
- Combien de noeuds a cet arbre ? Combien d'observations y a-t-il dans chaque noeud ? 

## Bagging

Le bagging (ou agrégation Bootstrap) est un type d'apprentissage d'ensemble dans lequel plusieurs modèles de base sont entrainés indépendamment et en parallèle sur différents sous-ensembles des données d'entraînement. Dans le bagging, les étapes d'entraînement et de prédiction sont précédées par une étape de bootstrap.

1. **Échantillonnage bootstrap** : Dans l'échantillonnage bootstrap, $K$ sous-ensembles aléatoires des données originales sont échantillonnés avec remplacement. Cette étape garantit que les modèles de base sont formés sur divers sous-ensembles des données, car certains échantillons peuvent apparaître plusieurs fois dans le nouveau sous-ensemble, tandis que d'autres peuvent être omis. Cela réduit les risques de surajustement et améliore la précision du modèle.

2. **Entraînement sur les modèles de base** : Après la première étape d'échantillonnage bootstrap, le modèle de base (arbres de décision, SVM...) est entraîné indépendamment sur chaque sous-ensemble de données bootstrap différent. Ces modèles de base sont généralement dits « faibles » car ils peuvent ne pas être très précis à eux seuls. 

3. **Agrégation** : Une fois que tous les modèles de base ont été entraînés, ils sont utilisé pour faire chacun une prédiction sur les données de test. Dans les modèles de classification, la prédiction finale est effectuée en agrégeant les prédictions des modèles de base en utilisant le vote majoritaire. Dans les modèles de régression, la prédiction finale est obtenue en faisant la moyenne des prédictions des modèles de base.

**Évaluation Out-of-Bag (OOB)** : Dans l'étape d'échantillonage, certaines observations sont exclues de l'échantillon boostrap. Ces observations *out-of-bag* peuvent être utilisées pour évaluer les performances du modèle.

![Classificateur par agrégation bootstrap](img/Bagging-classifier.png)

Les arbres de décision vus au point précédent sont extrêmement dépendant du training set. Les modèles de bagging, grâce à l'étape d'échantillonnage aléatoire, permettent de réduire cette sensibilité aux données d'entraînement.


### Exercice : Bagging à la main

Ecrire un bagging à la main de taille `K=100` *sur les données réduites* en suivant les étapes suivantes :

1. Créer K échantillons boostrap de même taille que l'échantillon d'entraînement :
    - Utiliser la fonction `np.random.choice` avec paramètre `replace=True` pour générer un tableau d'indices `indexes` de même taille que l'échantillon original
    - Un échantillon booststrap est donné par `X_train_S[indexes]`
2. Entraîner un arbre de décision simple (peu profond, avec un nombre d'observations minimal par feuille, ...) sur chaque échantillon bootstrap
3. Calculer la valeur prédite par bagging :
      - Calculer la prédiction de chaque arbre
      - Renvoyer la moyenne des valeurs prédites par les K arbres
4. Calculer et afficher les erreurs MAPE et RMSE *out-of-bag*. 

Une fois le modèle sur les données réduites maîtrisé, il est possible de passer sur les données complètes, tout en tenant compte des temps de calculs attendus.

### Exercice : Bagging de sklearn

- Comparer avec un bagging construit en utilisant la fonction `BaggingRegressor` du module `sklearn.ensemble`.
- Afficher le score *out-of-bag* de ce prédicteur.

### Random forest

Le forêts aléatoires sont une extension du bagging pour les arbres de décision qui permet de réduire davantage la variance du modèle, en ajoutant un caractère aléatoire lors de la création des arbres.

Pendant la construction d'un arbre, au lieu de rechercher la caractéristique la plus importante lors de la division d'un nœud, on recherche la meilleure caractéristique **parmi un sous-ensemble aléatoire de caractéristiques**. Il y a donc deux niveaux *random* dans ce modèle (d'où le nom "random forest"): le random du bootstrap et la sélection random des caractéristiques. Le premier réduit la dépendance au training set, le deuxième réduit la corrélation entre les arbres de la forêt. Il en résulte une grande diversité qui se traduit généralement par un meilleur modèle.

### Exercice

- A l'aide de la classe `RandomForestRegressor` du module `sklearn.ensemble`, entraîner une foret aléatoire avec paramètres par défaut sur les données réduites et comparer les erreurs avec les autres modèles. Afficher également le score *out-of-bag* de ce prédicteur.  
- Visualiser dans un graphe l'évolution des erreurs en fonction du nombre d'arbres de la forêt.
- Les résultats théoriques montrent que le bon nombre de caractéristiques à tenir en compte lors d'un split est soit de l'ordre de la racine carré du nombre total des caractéristiques, soit de son logarithme. Entraîner une foret aléatoire avec un nombre d'arbres choisi sur la base de la visualisation du point précédent, et un nombre de caractéristiques de l'ordre de la racine carré du nombre total de caractéristiques. Afficher également le score *out-of-bag* de ce prédicteur.  

  

### Extra trees

Proche des forêts aléatoires, les Extra Trees, dit aussi Extremely Randomized Trees, ajoutent une couche supplémentaire d'aléatoire au forêts, en choisissant le split d'une caractéristique au hasard, au lieu qu'en sélectionnant le meilleur split. Cela réduit par ailleurs de façon considérable le temps de calcul.

### Exercice

- Combiner la classe `ExtraTreeRegressor` et `BaggingRegressor` pour obtenir une forêt d'arbres extrêmement aléatoires.  
- Entraîner, prédire et afficher le score *out-of-bag* d'abord sur les données réduites, et ensuite sur les données complètes.

## Boosting

Nous avons vu que les méthodes de Bagging permettent de construire un prédicteur robuste à partir de plusieurs prédicteurs faibles. Les prédicteurs faibles peuvent être construits en parallèle, c'est-à-dire que la construction de chacun d'entre eux peut être faite indépendamment des autres.

Une autre technique d'ensemble, différente du Bagging, consiste à générer les prédicteurs de façon séquentielle, en améliorant à chaque itération le nouveau modèle par rapport au précédent. C'est la technique du **Boosting**.

Tout d'abord, un modèle est construit à partir des données d'apprentissage. Ensuite, un deuxième modèle est construit pour tenter de corriger les erreurs présentes dans le premier modèle. Cette procédure se poursuit et des modèles sont ajoutés jusqu'à ce que l'ensemble des données d'apprentissage soit prédit correctement ou que le nombre maximum de modèles soit atteint. 

Dans toutes les techniques de Boosting, des poids sont associés aux données d'apprentissage, et à chaque itération ces poids sont réajustés, de façon à augmenter le poids des données mal prédites  et réduire celui des données correctement prédites. Ceci permet au modèle suivant de se "concentrer" sur les mauvaise prédictions et de s'améliorer.

![Boosting classifier](img/Boosting-classifier.png)

Il existe plusieurs algorithmes de Boosting, parmi les plus connus nous pouvons citer : 
- **Gradient Boosting** :
    - Chaque nouveau modèle est entraîné pour minimiser la fonction de perte (erreur quadratique moyenne ou cross-entropie) du modèle précédent à l'aide d'une descente de gradient.
    - À chaque itération, l'algorithme calcule le gradient de la fonction de perte par rapport aux prédictions de l'ensemble actuel, puis entraîne un nouveau modèle faible pour minimiser ce gradient.
    - Les prédictions du nouveau modèle sont ensuite ajoutées à l'ensemble, et le processus est répété jusqu'à ce qu'un critère d'arrêt soit rempli.
- **AdaBoost** -- Adaptative Boosting
- **XGBoost** -- eXtreme Gradient Boosting



### Exercice

Utiliser le module `GradientBoostingRegressor` pour prédire la consommation via une regression par Gradient Boosting avec un learning rate égal à 0.7, un nombre d'estimateurs égal à 100 et une profondeur de l'arbre égale à 10. 

Si vous en avez le temps, faites de la validation croisée sur le learning rate et le nombre d'estimateurs.


## Feature importance/importance des variables

L'importance des variables indique dans quelle mesure chaque caractéristique contribue à la prédiction du modèle. Une première idée de l'importance des variables, par exemple, peut être donnée par la correlation entre chaque variable et la variable à prédire. Selon le modèle, il existe d'autres mesures qui peuvent aider à affiner la compréhension de quelles variables contribuent le plus à la prédiction.

### Exercice : Regressions linéaires

Combien de caractéristiques sont retenues dans les modèles Lasso ? 

Commenter au vu des erreurs obtenues.

### Exercice : Modèles d'ensembles 
Pour les arbres de décisions, les forêts aléatoires ou le gradient boosting, l'importance d'une variable peut être calculée comme la réduction totale du critère apportée par cette variable.

Afficher dans un histogramme à barres horizontales (`barh`) les 10 variables le plus importantes.

# Approche LSTM

Le but est d'illustrer l'utilisation d'un réseau de neurones récurrents (RNN) pour la prédiction de la consommation. Pour simplifier on oublie les autres variables et on cherche une relation entre les consommations passées (sur une période de temps à définir) et la consommation à venir (ou les consommations). 

On note $(y_k)_{k = 1, \dots,n}$ la série des consommations aux instants $t_k$ (les différentes demi-heures). 

Fixons $\tau \ge 1$ une taille de fenêtre de temps et $k \ge \tau$. On cherche une relation entre la consommation $y_{k+1}$ à l'instant $t_{k+1}$ et les consommations passées $y_{k-\tau}, \dots, y_k$ c'est à dire construire une prédiction $\hat y_{k+1}$ comme fonction de $y_{k}, \dots, y_{k-\tau}$
\begin{equation*}
    \hat y_{k+1} = g^{(\tau)} \big( y_{k}, \dots, y_{k-\tau} \big).
\end{equation*}
L'idée d'un RNN est d'introduire un état caché $h$ vecteur de $\mathbf{R}^d$ et une fonction paramétrique $f_\theta$ tels que,
\begin{equation*}
    \forall j \ge {k-\tau}, \quad h_j = f_\theta\big(h_{j-1}, y_j \big), 
\end{equation*}
avec $h_{k-\tau-1}$ fixé, par exemple un vecteur nul. Si on "dérécursive" cette relation on obtient 
\begin{align*}
    h_{k-\tau} &= f_\theta\big(h_{k-\tau-1}, y_{k-\tau} \big) \\
    h_{k-\tau+1} &= f_\theta\big(f_\theta\big(h_{k-\tau-1}, y_{k-\tau} \big), y_{k-\tau+1} \big) \\
    &\dots \\
    h_k & = f_\theta\big(f_\theta\big(\dots, y_{k-1} \big), y_{k} \big)
\end{align*}
Ainsi $h_k$ est une fonction de $\big(y_{k-\tau},\dots,y_k\big)$. Comme prédiction $\hat y_{k+1}$ on choisit alors une application linéaire de $h_k$
\begin{equation*}
    \hat y_{k+1} = A h_k + \beta
\end{equation*}

In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary

## Préparation des données 

Il est important de normaliser les données puis de créer un dataset que l'on va séparer en 2 jeux disjoints: `train set` et `test set`.  

1. Normaliser les données et créer un tensor `pytorch`
2. Fixer une taille de fenêtre $\tau$ (par exemple $\tau = 92$ pour avoir 2 jours d'historique) et découper la série temporelle pour créer un dataset
\begin{equation*}
    \mathcal{D} = \big\{ \forall k \ge \tau+1, \quad \big( (y_{k-\tau},\dots,y_k), y_{k+1} \big) \big\}
\end{equation*}

3. Découper le dataset $\mathcal{D}$ en des données d'entrainement et des donnés de test.
4. Créer un `dataloader` `pytorch` pour charger les données par paquet (batch) de taille 32.  

## Initialisation de la fonction de prédiction 

Créer une classe (fille de `nn.Module` de `pytorch`, on verra le détail demain dans le TP3) qui implémente la fonction $g^{(\tau)}$.

## Entrainement (apprentissage) de la fonction de prédiction

On va parcourir plusieurs fois le `train set` pour apprendre la fonction paramétrique $g^{(\tau)}$: apprendre c'est à dire résoudre un problème d'optimisation. Le problème ici est de minimiser un critère: par exemple la distance entre la valeur prédite $g^{(\tau)}(y_k, \dots, y_{k-\tau})$ et $y_{k+1}$.  

## Une première vérification

Pour se comparer on utilise un estimateur naturel (pour ce problème d'estimation de la consommation) qui consiste à utiliser la consommation actuelle $y_k$ pour prédire la consommation de la demi-heure suivante $y_{k+1}$. 

On utilise l'objet `transformer` pour revenir dans l'échelle des consommations et on calcule la MAPE pour ces deux estimateurs. 

## Quelques prédictions 

On fait quelques expériences numériques pour visualiser notre fonction de prédiction basée sur le LSTM entrainé. Attention on fait ici juste une illustration de quelques scénarios, il faudrait une étude plus quantitative pour "valider" ou non le modèle.